# Unidad 04
## Ejercicios

In [1]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\joi\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [3]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

Kernel added: #!sql-Pampero

1.	Escriba una consulta sobre la vista ValoresPedido que retorne, por cada cliente y orden, el valor promedio de movimiento de los tres últimos pedidos del cliente.

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, val,
  AVG(val) OVER(PARTITION BY IDCliente
                ORDER BY FechaPedido, IDPedido
                ROWS BETWEEN 2 PRECEDING
                        AND CURRENT ROW) AS promovil
FROM ValoresPedido;


2.	Escriba una consulta sobre la tabla Pedidos, y filtre los tres pedidos con los valores de flete más altos por cada distribuidor (EnvioPor) utilizando IDPedido como desempate.

In [ ]:
USE Pampero;
WITH C AS
(
  SELECT EnvioPor, IDPedido, Flete,
    ROW_NUMBER() OVER(PARTITION BY EnvioPor 
                      ORDER BY Flete DESC, IDPedido) AS rownum
  FROM Pedidos
)
SELECT EnvioPor, IDPedido, Flete
FROM C
WHERE rownum <= 3
ORDER BY EnvioPor, rownum;


3.	Consulte la vista ValoresPedido. Necesita calcular la diferencia entre el valor del pedido actual y el valor del pedido previo del cliente, además a la diferencia entre el valor de pedido actual y el valor de pedido siguiente del cliente.

In [ ]:
SELECT IDCliente, IDPedido, FechaPedido, val, 
  val - LAG(val) OVER(PARTITION BY IDCliente 
                      ORDER BY FechaPedido, IDPedido) AS difprev,
  val - LEAD(val) OVER(PARTITION BY IDCliente 
                       ORDER BY FechaPedido, IDPedido) AS difsig
FROM ValoresPedido;


4.	Calcular la cantidad total de pedidos agrupados por IDEmpleado, mostrando si los pedidos fueron despachados antes, en la fecha o después de la FechaRequerida. Resolver de dos maneras, una utilizando SUM y CASE y otra utilizando el operador PIVOT

In [ ]:
SELECT
    IDEmpleado,
    SUM(CASE WHEN FechaEnvio < FechaRequerida THEN 1 ELSE 0 END) AS Despachado_Anticipado,
    SUM(CASE WHEN FechaEnvio = FechaRequerida THEN 1 ELSE 0 END) AS Despachado_A_Tiempo,
    SUM(CASE WHEN FechaEnvio > FechaRequerida OR FechaEnvio IS NULL THEN 1 ELSE 0 END) AS Despachado_Tardio
FROM Pedidos
GROUP BY IDEmpleado
ORDER BY IDEmpleado;

SELECT IDEmpleado, Despachado_Anticipado, Despachado_A_Tiempo ,Depachado_Tardio
FROM (SELECT IDPedido, IDempleado, 
CASE WHEN FechaEnvio < FechaRequerida THEN 'Despachado_Anticipado'
     WHEN FechaEnvio = FechaRequerida THEN 'Despachado_A_Tiempo'
     ELSE  'Depachado_Tardio' END AS Despacho FROM Pedidos) AS D
PIVOT (COUNT(IDPedido) FOR Despacho IN (Despachado_Anticipado, Despachado_A_Tiempo ,Depachado_Tardio)) AS P
ORDER BY IDEmpleado;


5.	Muestre la suma total de Flete (flete) para cada IDEmpleado, con los años de pedido (2019, 2020, 2021) como columnas.

In [ ]:
SELECT IDEmpleado, [2019], [2020], [2021]
FROM (SELECT IDEmpleado, YEAR(FechaPedido) AS Anio, Flete FROM Pedidos) AS D
PIVOT (SUM(Flete) FOR Anio IN ([2019], [2020], [2021])) AS P
ORDER BY IDEmpleado;


6.	Calcule la suma total de Flete de los pedidos. Muestre totales intermedios por jerarquía, que incluyan el total por año, el total por año/mes, y un total general.

In [ ]:
SELECT
    YEAR(FechaPedido) AS anio,
    MONTH(FechaPedido) AS mes,
    SUM(Flete) AS total_flete
FROM Pedidos
GROUP BY ROLLUP(YEAR(FechaPedido), MONTH(FechaPedido)) -- Agrupamiento jerárquico
ORDER BY anio, mes;


7.	Calcule la suma total de Flete de los pedidos para los siguientes tres conjuntos de agrupamiento no jerárquicos:
1. Agrupado por PaisEnvio y Transportista.
2. Agrupado solo por PaisEnvio.
3. Un total general.

In [ ]:
SELECT
    PaisEnvio,
    T.NombreEmpresa,
    SUM(Flete) AS total_flete
FROM Pedidos P JOIN Transportistas T ON T.IDTransportista = P.EnvioPor
WHERE PaisEnvio LIKE 'A%'
GROUP BY GROUPING SETS (
    (PaisEnvio, T.NombreEmpresa), -- Agrupamiento 1
    (PaisEnvio),          -- Agrupamiento 2
    ()                    -- Agrupamiento 3 (Total General)
)
ORDER BY PaisEnvio, T.NombreEmpresa;
